In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train=pd.read_csv('mnist_train.csv',header=None)



In [3]:
test=pd.read_csv('mnist_test.csv',header=None)


In [4]:
x_train=train.iloc[:,1:].values
y_train=train.iloc[:,0].values
x_test=test.iloc[:,1:].values
y_test=test.iloc[:,0].values

In [5]:
#choosing right number of features
svdtrain=TruncatedSVD(n_components=100)
x_train_reduced=svdtrain.fit_transform(x_train)
x_test_reduced=svdtrain.transform(x_test)
cumsum = np.cumsum(svdtrain.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.90) + 1 
print ('The number of required components to get variance of 90 % is',d)
print('The shape of the reduced data',x_train_reduced.shape)

The number of required components to get variance of 90 % is 87
The shape of the reduced data (60000, 100)


In [6]:
# reduced dimensionality datasets
x_train_data= StandardScaler().fit_transform(x_train_reduced)
x_test_data= StandardScaler().fit_transform(x_test_reduced)
y_train_data=y_train
y_test_data=y_test
print(x_train_data)
print(y_train_data.shape,y_test_data.shape)

[[ 0.33128275  0.00928816 -0.62778574 ...  0.51768097  0.2672156
   0.68878891]
 [ 0.54743313  1.85805826 -0.62736201 ... -0.49372053 -0.17292538
  -0.07518921]
 [-1.58753472  0.73329841  0.76166989 ... -0.12899302  1.35534096
   1.99294428]
 ...
 [-0.57767137 -0.1687613   0.32507627 ... -0.28660401 -0.48216423
  -0.05098249]
 [-0.87512842  0.71491174 -0.03275423 ...  0.47895012 -0.37665666
   1.94964964]
 [-0.757626   -0.15292631 -0.04610125 ...  2.32102863 -0.48514614
   0.02404884]]
(60000,) (10000,)


In [7]:
# accuracy using logistic regression using svd
#model fitting 
model=LogisticRegression(max_iter=10000)
model.fit(x_train_data,y_train_data)
print(model.score(x_test_data,y_test_data))

0.9211


In [8]:
# accuracy using KNN Classifier using svd
model = KNeighborsClassifier(n_neighbors=3)
model.fit(x_train_data,y_train_data)
print(model.score(x_test_data,y_test_data))


0.9417


In [23]:
z_train=np.concatenate((x_train_data,x_test_data))
z_test=np.concatenate((y_train_data,y_test_data))
print(z_train.shape)

(70000, 100)


In [21]:
from numpy import mean
from numpy import std
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
steps = [('svd', TruncatedSVD(n_components=100)), ('m', LogisticRegression())]
model = Pipeline(steps=steps)
# evaluate model
n_scores = cross_val_score(model, z_train, z_test, scoring='accuracy',  n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores)))

ValueError: n_components must be < n_features; got 100 >= 100